<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/PySparkRFClassificationDogFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 64.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4f1df49de5361db53b04f5151baa4a1a79dc28f6ea72fce2af575b611f4afeec
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()

dog food company to try : predict why some batches of dog food are spoiling much quicker than intended

the amounts of the five preservative chemicals used can vary a lot, but which is the chemical that has the strongest effect? 

While manufactoring first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. 

Find out which one out of A,B,C, or D preservatives is causing the problem

Use RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! 

create a model and then find out how you can decide which chemical is the problem!

Pres_A : Percentage of preservative A in the mix

Pres_B : Percentage of preservative B in the mix

Pres_C : Percentage of preservative C in the mix

Pres_D : Percentage of preservative D in the mix

Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [3]:
data = spark.read.csv('dog_food.csv', inferSchema = True, header = True)

In [4]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [16]:
data.head(2)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)]

In [6]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol="features")

In [8]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.classification import RandomForestClassifier

In [11]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')

In [12]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
final_data = output.select('features','Spoiled') # only these two required by model to fit 

In [14]:
rfc_model = rfc.fit(final_data)

many machine learning models produce some sort of coefficient value for each feature involved, indicating their importance or predictive power 

Tree classifiers have a .featureImportance attribute available 

It can be used to check which feature (preservative) is causing the spoilage 

In [15]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0134, 1: 0.0101, 2: 0.9527, 3: 0.0238})

first entry of SpareVector is number of features used 
second entry is dictionary of feature number & the importance 

feature index 2 which is C is most important 
